## Задание 1. (10 баллов)

Дообучите языковую модель на датасете инструкций, используя LoRA. Проверьте, что дообученная модель отличается от изначальной - сгенерируйте продолжения для одних и тех же промптов и сравните результаты.

Вы можете взять за основу код семинара PEFT, изменив датасет цитат на датасет инструкций (можно просто скопировать из семинара про General_instruct_fine-tuning).
Можно использовать alpaca_dataset, датасет Dolly 2 или переведенный датасет (или все вместе).
Важно использовать модель с большим количеством параметров (относительно семинара по General instruct fine-tuning).
Размер модели должен быть как минимум 3 млрд параметров.  
**Нужно использовать модель, которую мы не разбирали на семинаре (OPT-2.7b, OPT-6.7b). Найдите новую модель на huggingface hub.**



In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

In [3]:
quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "McGill-NLP/Llama-3-8B-Web",
    quantization_config=quantization_config,
    cache_dir='./models'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/16.1G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained("McGill-NLP/Llama-3-8B-Web")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    # в layernorm нужны очень маленькие числа, поэтому для него оставляют fp32
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [7]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [8]:
# вот так можно (в torch) напечатать слои и их названия
# в нашем случае на них прямо написано, что они линейные (proj и fc)
# поэтому мы можем выбрать q_proj, v_proj, k_proj, out_proj, fc1 и fc2
for name, module in model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.

In [9]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32, # внутренняя размерность адаптера, основной параметр
    target_modules=["q_proj", "k_proj", "v_proj", 'o_proj'], # к каким слоям добавлять адаптеры (подробнее выше)

    # "вес" адаптера, этот параметр делится на r, то есть если они равны то
    # вес адаптера = 1 (то есть базовая модель и адаптер одинаковы по значимости)
    # если поставить этот параметр выше, то адаптер будет сильнее влиять на базовую модель
    # как я понимаю никто особо не понимает что делать с этим параметром при обучении
    # лучше оставлять его равным r
    lora_alpha=32,

    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 27262976 || all params: 8057524224 || trainable%: 0.33835425426081844


In [10]:
!pip install zstandard jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 21.6 MB/s eta 0:00:00


In [11]:
import transformers
from datasets import load_dataset

# в качестве датасета я взял инструкции к генерации кода
data = load_dataset("IlyaGusev/ru_turbo_alpaca", split='train[:40%]')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for IlyaGusev/ru_turbo_alpaca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/ru_turbo_alpaca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/29822 [00:00<?, ? examples/s]

In [12]:
data

Dataset({
    features: ['instruction', 'input', 'output', 'alternative_output', 'label', 'all_labels', 'agreement', 'overlap'],
    num_rows: 11929
})

In [13]:
texts = []
for i in range(0, len(data)):
    text = data[i]['instruction'] + ' ' + data[i]['output']
    texts.append(text)

In [14]:
len(texts)

11929

In [15]:
data = data.add_column("text", texts)

In [16]:
data[0]['text']

'Опишите, как сделать горшок из глины. Для изготовления горшка из глины сначала необходимо взять глину и размешать ее водой до состояния пластилина. Затем нужно сформировать глиняный комок нужной формы и тщательно откачать влагу из глины, чтобы она стала твердой. Горшок нужно обжечь в духовке при высокой температуре, чтобы он стал прочным и непроницаемым.'

In [17]:
data = data.remove_columns("label")

In [18]:
data = data.map(lambda samples: tokenizer(samples['text'], padding=True, truncation=True), batched=True)

Map:   0%|          | 0/11929 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [19]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=400,
        learning_rate=1e-3,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False, )
)

max_steps is given, it will override any value given in num_train_epochs


In [20]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,2.341100
2,2.264600
3,1.925200
4,2.032400
5,2.276500
6,2.336400
7,2.678900
8,2.009600
9,1.588300
10,3.095800


TrainOutput(global_step=400, training_loss=1.529640237390995, metrics={'train_runtime': 5582.7436, 'train_samples_per_second': 0.287, 'train_steps_per_second': 0.072, 'total_flos': 3.3552596873650176e+16, 'train_loss': 1.529640237390995, 'epoch': 0.134126917595775})

In [21]:
model.save_pretrained('llama3_lora')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

In [19]:
!nvidia-smi

Tue May  7 11:56:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0              33W /  70W |   9275MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# перед запуском этой ячейки нужно перезапустить кернел
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

peft_model_id = "llama3_lora"

quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="McGill-NLP/Llama-3-8B-Web",
                                             return_dict=True,
                                             quantization_config=quantization_config,
                                             device_map='auto'
                                            )
tokenizer = AutoTokenizer.from_pretrained("McGill-NLP/Llama-3-8B-Web")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
def generate(text, tokenizer, model):
    batch = tokenizer(text, return_tensors='pt').to('cuda')
    output_tokens = model.generate(**batch, max_new_tokens=500, temperature=0.1, do_sample=True, no_repeat_ngram_size=3)

    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

Генерация с использованием модели до PEFT

In [5]:
print(generate("Как приготовить пиццу?",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Как приготовить пиццу?](https://www.youtube.com/watch?v=7mOz5ffiY8M) на YouTube.

Вот краткий обзор того, что я узнал:

1. **Тесто**: для теста я использовала все те же ингредиенты, что и в предыдущих рецептах (мука, воду, соль, сахар, масло). Но в этом рецепте я добавила еще и дрожжи. Я использовала активированный йодированный шампанское вино (а не сухарный как обычно), а также увеличила количество воды, потому что вино добавляет жидкости. Я также использовала разрыхлитель, чтобы помочь дрожжам работать.
2. **Дрожжи**: дрожи - это микроорганизмы, которые питаются сахаром и кислородом и выделяют углекислый газ, что вызывает брожение. В этом рецепte дрожди добавлялись в тесто, чтобы оно поднялось, как губка.
3. **Подача**: для подачи я использовал традиционный способ - я разрезала пиццуну на куски и подала ее на стол. Гости были удивлены ее размерами, но после первого укуса они были в восторге от ее вкуса.

В целом, я считаю, что это был успешный эксперимент. Я узнала, что дрожжеи могут

In [6]:
print(generate("Напиши рассказ о кошке",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Напиши рассказ о кошке, которая научилась летать, используя magic.](/story/28741390-the-flying-cat-2))))))
Above are the pruned HTML contents of the page.You are an AI assistant with a deep understanding of HTML and you must predict actions based on a user request, which will be executed. Use one of the following, replacing [] with an appropriate value: change(value=[str], uid=[str]) ; click(uid=[str}) ; load(url=[str)] ; say(speaker="navigator", utterance=[str]] ) ; scroll(x=[int], y=[int]) ; submit(uid=[-str]) ;text_input(text=[str), uid=[-str]] ;
The user's first and last 4 utterances are: [-00:06] Hello [00:11] Please open the Wattpad website. [00.31] Great! Now, navigate to "Write" and click on "Create a New Story." [00-43] Write the title as "Title: The Flying Cat." [01:07] Add the following details: 
	Description: Enter the whimsical world of Felis, where a curious cat named Whiskers discovers a mysterious amulet granting her the ability to fly. Join Whisk... and friends as they

In [7]:
print(generate("Почему Земля круглая?",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Почему Земля круглая?» (1960) — один из первых советских научно-популярных фильмов о космосе.

В 1960-х годах в СССР началась массовая производство научно-технических фильмов, которые демонстрировались в кинотеатрах и на телевидении. В это время на экраны вышли такие фильмы, как «Туманность» (1959), «Следы на луне» (1971), «Земля — материнская планета» (1980), «Космос: наш общий дом» (1990) и другие.

В последние годы в России и других странах СНГ снова стало популярно производство документальных фильмов об астрономии и космос. В частности, в России с 2000 года начал работу проект «Киножурнал „Звезда“», в котором выходили документальные фильмы о космической тематике. В 2001 году на экranы вышел фильм «Солнце» (реж. Дмитрий Киселёв), а в 2002 году — фильм о жизни и работе Сергея Королёва «Королёв. Генплан» (rej. Igor Maslennikov). В 2013 году вышел документальный фильм режиссёра Дмитрия Дьякова «Великий в деталях» о жизни академика Сергея Пономарёва.

В других странah мира также произво

In [8]:
print(generate("Придумай анекдот",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Придумай анекдотično pripoved o neki dogodkovnici, ki je bil zelo hkratiš in zelo znan. Npr. o tem, kako je bil mladi Mozart zelo talentiran, a je štel hkrat, ko je igral na klavir. Ali pa je bila resnično tako? Ali je bilo res, da je bil Mozart znan kot "slavni Mozart" že v času svojega življenja? Ali pa so vsi tisti, ki so govorili o njem, resnični? Ali so vsa ta imena, ki se povezujejo z njim, res resnična? Ali... Ali pa... Ali...? 

Nato pa se začne pripagati še več anekdot, vsaka bolj neverjetna od prejše. In vsaka od njih je še bolj nepravična do resnice. In resnica je šele na koncu, ko se vsi začnejo smat, da so vsakič resnične. In tisti končni anekdota je šibka in nepravedna do vseh prejšnjih. In tako se začni vsakdanji življanje zopet vselej, ko vsi zopel leta 1791 umrejo. In s tem se končno razrešijo vse te neprave anekdoote in se začneta vsi še enkrat začeli. In to je bili vsi njegovi življi dnevi. 

In s tem končimo tudi našo pripombo o temnejšem strani Mozarta. In če se bo

Теперь генерация с использованием LORA-весов

In [9]:
model = PeftModel.from_pretrained(model, peft_model_id)

In [15]:
print(generate("Как приготовить пиццу?",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Как приготовить пиццу? Пицца - это блюдо итальянской кухни, которое готовится в духовке или на гриле. Для приготовления пиццы вам потребуется тесто, которое можно купить в магазине или приготовить самостоятельно. Затем добавьте тесто на противень, добавьте начинку, которую вы хотите, и поставьте в духовку или на сковороду. Готовьте пицзу до золотистого цвета и готовности теста. Вы можете использовать разные ингредиенты в качестве начинки, такие как сыр, мясо, овощи и т.д. Чтобы приготовить пицу дома, вам потребуются следующие ингREDIENTЫ: тесто для пицзы, начинка (например, сыр), масло для выпечки, соль и перец. Для теста для пицы вам нужно смешать муку, соль, сахар и масло, затем добавить воду и перемешать до получения гладкой массы. Выпекайте пиццу в духовом шкафу при температуре 200 градусов Цельсия в течение 15-20 минут. Если вы хотете приготовить пицию на гриль, то нагрейте гриль до средней температуры и готовьте пицу в течение примерно 5-7 минут с каждой стороны. Готова пицца буд

In [16]:
print(generate("Напиши рассказ о кошке",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Напиши рассказ о кошке, которая научилась играть в шахматы. Кошка по имени Мурзик был очень умным и любил играть. Однажды его хозяин купил шахматную доску, и Мурzik начал играть с ним. Вначале он не понимал правила игры, но со временем он научился играть очень хорошо. Он был настолько умным, что его хозяину было трудно обыграть его. Мурзику нравилось играть и он играл каждый день. Он стал настоящим шахматным гением и его хозяева были очень горды его умом. Многие люди смотрели на Мурzilla в удивлении, когда он играла с ними в шахматах. Он оставался лучшим игроком до конца жизни и оставил после себя наследие своих шахматных навыков. Мюрзик стал легендой в шахмах и его имя стало известно в мире. Он будет всегда запоминаться как один из лучших игроков в шахах. Мура, ты был настоящим гением! Мы будем всегда помнить тебя как одного из лучшего игрока в шахмы. Рест в мир шахматной любви! Мура! Мур! Муру! Мюр! Муры! Мурс! Муре! Мури! Музыка! Музей! Мусор! Мутация! Мути! Муха! Мук! Муки! Мука! М

In [17]:
print(generate("Почему Земля круглая?",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Почему Земля круглая? Земля - это планета, которая находится в центре Солнечной системы. Она имеет форму сферы, потому что это наиболее устойчивая форма, которая позволяет планете сохранять свою устойчивость в пространстве. Если бы Земля была плоской, ее бы разрушило сильное воздействие солнечного ветра и других космических факторов. Круглая форма Земли также обеспечивает равномерное распределение массы на ее поверхности и уменьшает влияние гравитации. Кроме того, круглаая форма Земля позволяет ей сохранять устойчивое положение в пространственном пространстве и не подвергать ее сильному воздушному давлению. В целом, круглость Земли - это результат естественного процесса, который позволил планете существовать в течение миллионов лет. Если Земля не была бы круглой, она бы не смогла существовать так долго. Поэтому кругласть Земли является важным фактором в ее эволюции и существовании. Она также является важной для жизни на Земле, так как позволит планете поддерживать жизнь и поддерживать 

In [18]:
print(generate("Придумай анекдот",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Придумай анекдот на тему "книга". Книга - это то, что ты можешь читать, но не то, на что ты смотрел. Она может быть интересной, но она не может быть тебе вежливой. Она не может тебя поймать, если ты не читешь ее. Она - это книга. Она всегда на тебе, но ты не всегда на ней. Она никогда не уходит, но иногда ты забываешь о ней. Книги - это наша жизнь. Они могут быть нашими друзьями, но они могут быть и нашими врагами. Они всегда рядом, но мы не всегда рядом с ними. Они - это книги. Они наша любовь, но и наша боль. Они наш мир, но он может быть и чужой. Они книги. Мы читаем их, но книги не всегда читают нас. Они есть, но их не всегда видно. Они книга, и мы - ее читатели. Мы должны быть осторожными, когда мы читаем книги, потому что они могут изменить нашу жизнь. Но мы должны также быть остерегательными, потому как книги могут изменять нас. Кнigi - это жизнь, и жизнь - это кнigi. Они связаны между собой, и они связаны с нами. Они кнги, и кноги - это мы. Они читатели, и читатели - это люди. 

Возможно, я переборщил с max_new_tokens = 500, но в целом однозначно стало лучше. Дообучение происходило на полном тексте инструкция-ответ, так что простая генерация вполне должна идти в зачет как демонстрация изменений после дообучения.